![](fraud.png)


<div class="download-buttons" style="text-align: center;">
  <style>
    .download-buttons a.button {
      display: inline-block;
      padding-right: 10px;
      padding-left: 35px;
      padding-top: 10px;
      padding-bottom: 10px;
      margin-right: 25px;
      background-color: #343a40;
      color: #fff;
      text-decoration: none;
      border-radius: 15px;
      position: relative;
    }

    .download-buttons a.button:last-child {
      margin-right: 0;
    }

    .download-buttons a.button:hover {
      background-color: #55595c;
    }

    .download-buttons a.button:after {
      content: "\2193";
      font-size: 20px;
      position: absolute;
      top: 50%;
      left: 12px; /* Added margin from the text */
      transform: translateY(-50%);
      border-bottom: 2px solid #fff;
      padding: 1px; /* Padding for the mini border block */
    }
  </style>

  <script>
    function createDynamicButton(href, text) {
      var button = document.createElement("a");
      button.href = href;
      button.className = "button";
      button.appendChild(document.createTextNode(text));
      return button;
    }

    var downloadButtonsDiv = document.querySelector(".download-buttons");

    var documentButton = createDynamicButton(
      "https://github.com/bcervantesalvarez/Portfolio/blob/main/posts/FraudML/index.qmd",
      "Download .qmd"
    );
    downloadButtonsDiv.appendChild(documentButton);

    var csvButton = createDynamicButton(
      "https://www.kaggle.com/datasets/harithapliyal/card-transaction-log/download?datasetVersionNumber=1",
      "Download .zip"
    );
    downloadButtonsDiv.appendChild(csvButton);
  </script>
</div>


## Abstract

Fraud detection is a challenging task, but advancements are being made each year. This project aims to improve fraud detection using a Gradient Boosting Machine (GBM) model. The dataset is prepared, and the model is trained and evaluated on validation and test sets. Performance metrics such as accuracy, precision, recall, AUC, and Kappa coefficient are computed. A comparison is made with the existing 'isFlaggedFraud' column model. The GBM model demonstrates higher performance in various aspects, highlighting its superiority in classifying fraudulent transactions. However, further improvements are needed to enhance fraud detection while maintaining precision.

## Introduction

Fraudulent activities pose a significant threat in various industries, including financial transactions. Detecting fraudulent transactions is a complex task, but advancements in machine learning techniques have shown promise in improving accuracy and precision. In this project, we focus on enhancing fraud detection using a Gradient Boosting Machine (GBM) model.

## Methodology 

First, we set up the environment by importing necessary libraries and loading the dataset. The dataset, named 'Card-Transaction_log.csv', contains relevant information for training and evaluation. We explore the dataset by printing the first few rows and examining its shape.

Next, we perform data wrangling by applying one-hot encoding to the features, excluding 'isFraud', 'nameOrig', 'nameDest', and 'isFlaggedFraud' columns. This process prepares the data for training the GBM model.

To evaluate the model's performance, we split the data into training, validation, and test sets. The random seed is set for reproducibility. The GBM model is trained on the training set using the specified parameters, such as loss function, learning rate, number of estimators, and maximum depth. The model's training time is recorded for analysis.

We then evaluate the GBM model on the validation set to measure its accuracy, precision, recall, confusion matrix, AUC, and Kappa coefficient. These metrics provide insights into the model's performance in classifying fraudulent and non-fraudulent transactions.

Further, we assess the GBM model's performance on the test set and compute the corresponding metrics. The test accuracy, precision, recall, confusion matrix, AUC, and Kappa coefficient provide a comprehensive evaluation of the model's ability to classify transactions accurately.

To compare the GBM model with the existing 'isFlaggedFraud' column model, we calculate the accuracy, precision, and recall scores for the 'isFlaggedFraud' column. This analysis allows us to assess the GBM model's superiority in detecting fraudulent transactions.

Based on the results, we discuss the GBM model's performance, highlighting its accuracy, precision, recall, AUC, and Kappa coefficient. We analyze the confusion matrix to identify areas where the model misclassifies transactions. While the GBM model outperforms the 'isFlaggedFraud' column model in several aspects, it still has room for improvement in detecting fraudulent instances while maintaining high precision.


::: {.callout-caution}
## Caution

Ensure you are using the correct Python version: Python 3.10.9 ('base') ~/anaconda3/bin/python
:::


### Set up

In [1]:
import numpy as np
import pandas as pd
import time
from sklearn.ensemble import GradientBoostingClassifier  # For classification tasks
from sklearn.ensemble import GradientBoostingRegressor  # For regression tasks
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    precision_score,
    recall_score,
    roc_auc_score,
    roc_curve,
    cohen_kappa_score,
)

ds = pd.read_csv('Card-Transaction_log.csv')

# Print the first 5 rows
print(ds.head(5))

# Print the shape of the frame
print(ds.shape)

   step      type    amount     nameOrig  oldbalanceOrg  newbalanceOrig  \
0     1   PAYMENT   9839.64  C1231006815       170136.0       160296.36   
1     1   PAYMENT   1864.28  C1666544295        21249.0        19384.72   
2     1  TRANSFER    181.00  C1305486145          181.0            0.00   
3     1  CASH_OUT    181.00   C840083671          181.0            0.00   
4     1   PAYMENT  11668.14  C2048537720        41554.0        29885.86   

      nameDest  oldbalanceDest  newbalanceDest  isFraud  isFlaggedFraud  
0  M1979787155             0.0             0.0        0               0  
1  M2044282225             0.0             0.0        0               0  
2   C553264065             0.0             0.0        1               0  
3    C38997010         21182.0             0.0        1               0  
4  M1230701703             0.0             0.0        0               0  
(6362620, 11)


### Data Wrangling 

In [2]:
X = pd.get_dummies(ds.drop(['isFraud', 'nameOrig', 'nameDest', 'isFlaggedFraud'], axis=1))   # Perform one-hot encoding on the features
y = ds['isFraud']

::: {.callout-note}
## Note

This dataset was already cleaned and tidy, so no additional wrangling was needed.
:::

### Split the data into training, validation, and test sets

In [3]:
# Set the random seed
np.random.seed(57)

# Training split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, random_state=42)


# Train the model
start_time = time.time()  # Start the timer

gbm = GradientBoostingClassifier(loss='log_loss', learning_rate=0.1, n_estimators=100, max_depth=3)
gbm.fit(X_train, y_train)

end_time = time.time()  # Stop the timer

elapsed_time = end_time - start_time
print(f"Elapsed Time: {elapsed_time} seconds")

Elapsed Time: 1052.879863023758 seconds


### Evaluate on the validation set

In [4]:
y_pred_val = gbm.predict(X_val)
accuracy_val = accuracy_score(y_val, y_pred_val)
precision_val = precision_score(y_val, y_pred_val)
recall_val = recall_score(y_val, y_pred_val)
confusion_matrix_val = confusion_matrix(y_val, y_pred_val)
auc_val = roc_auc_score(y_val, y_pred_val)
fpr_val, tpr_val, thresholds_val = roc_curve(y_val, y_pred_val)
kappa_val = cohen_kappa_score(y_val, y_pred_val)

print(f"Validation Accuracy: {accuracy_val}")
print(f"Validation Precision: {precision_val}")
print(f"Validation Recall: {recall_val}")
print("Validation Confusion Matrix:")
print(confusion_matrix_val)
print(f"Validation AUC: {auc_val}")
print(f"Validation Kappa: {kappa_val}")

Validation Accuracy: 0.9993241777758219
Validation Precision: 0.9693396226415094
Validation Recall: 0.4963768115942029
Validation Confusion Matrix:
[[635421     13]
 [   417    411]]
Validation AUC: 0.7481781765679447
Validation Kappa: 0.6562465275067701


### Evaluate on the test set

In [5]:
y_pred_test = gbm.predict(X_test)
accuracy_test = accuracy_score(y_test, y_pred_test)
precision_test = precision_score(y_test, y_pred_test)
recall_test = recall_score(y_test, y_pred_test)
confusion_matrix_test = confusion_matrix(y_test, y_pred_test)
auc_test = roc_auc_score(y_test, y_pred_test)
fpr_test, tpr_test, thresholds_test = roc_curve(y_test, y_pred_test)
kappa_test = cohen_kappa_score(y_test, y_pred_test)

print(f"Test Accuracy: {accuracy_test}")
print(f"Test Precision: {precision_test}")
print(f"Test Recall: {recall_test}")
print("Test Confusion Matrix:")
print(confusion_matrix_test)
print(f"Test AUC: {auc_test}")
print(f"Test Kappa: {kappa_test}")

Test Accuracy: 0.9992958875431819
Test Precision: 0.9455958549222798
Test Recall: 0.46085858585858586
Test Confusion Matrix:
[[635449     21]
 [   427    365]]
Test AUC: 0.7304127697259946
Test Kappa: 0.6193839067580158


### How does this compare to the current model?

In [6]:
# Comparison with 'isFlaggedFraud' column
is_flagged_fraud_accuracy = accuracy_score(ds['isFraud'], ds['isFlaggedFraud'])
is_flagged_fraud_precision = precision_score(ds['isFraud'], ds['isFlaggedFraud'])
is_flagged_fraud_recall = recall_score(ds['isFraud'], ds['isFlaggedFraud'])

print(f"Accuracy (isFlaggedFraud): {is_flagged_fraud_accuracy}")
print(f"Precision (isFlaggedFraud): {is_flagged_fraud_precision}")
print(f"Recall (isFlaggedFraud): {is_flagged_fraud_recall}")

Accuracy (isFlaggedFraud): 0.9987116942391656
Precision (isFlaggedFraud): 1.0
Recall (isFlaggedFraud): 0.0019481310118105442


## Results

Based on the evaluation of the GBM (Gradient Boosting Machine) model on the test set, the following performance metrics were obtained:

- Test Accuracy: 0.9992958875431819
- Test Precision: 0.9455958549222798
- Test Recall: 0.46085858585858586
- Test AUC: 0.7304127697259946
- Test Kappa: 0.6193839067580158

The GBM model achieved a high accuracy score of 0.999, indicating a strong ability to correctly classify transactions in the test set. With a precision of 0.946, the GBM model accurately identified approximately 94.6% of the predicted fraudulent transactions. However, the recall score of 0.461 suggests that the GBM model only captured around 46.1% of the actual fraudulent transactions.

Analyzing the confusion matrix, the GBM model correctly classified a large number of non-fraudulent transactions (true negatives) and a significant portion of fraudulent transactions (true positives). However, there were some instances where the GBM model misclassified non-fraudulent transactions as fraudulent (false positives) and failed to identify certain fraudulent transactions (false negatives).

The AUC score of 0.730 indicates a moderate level of discrimination between fraudulent and non-fraudulent transactions for the GBM model. Although the GBM model demonstrates substantial agreement beyond chance with a Kappa coefficient of 0.619, there is room for improvement in its ability to detect fraudulent instances while maintaining high precision.

### Comparison with Original Model

Comparing the GBM model with the 'isFlaggedFraud' column model, the GBM model outperforms in several key aspects. It achieves higher accuracy, precision, recall, and AUC scores, indicating superior overall performance in classifying fraudulent transactions. The 'isFlaggedFraud' column model, although having perfect precision, only detects a very small number of actual fraud cases, resulting in low recall.

## Limitations and Further Study

It is important to consider the specific goals and requirements of the application. While the GBM model provides better overall performance, it may have a higher number of false positives compared to the 'isFlaggedFraud' column model. Further enhancements can be made to improve the GBM model's ability to detect fraudulent transactions while maintaining high precision.

These results provide valuable insights into the GBM model's performance on unseen data, indicating its generalization capability. It is crucial to consider the test set performance as the final evaluation of the GBM model's effectiveness in real-world scenarios.
